# Git & Github

詳しい説明は、オンラインで読める[ProGit](https://git-scm.com/book/ja/v2)を参照のこと。  
他、[サルでもわかるGit入門](https://backlog.com/ja/git-tutorial/)、[Attlasian Gitチュートリアル](https://ja.atlassian.com/git/tutorials)、[Udemy Git入門](https://www.udemy.com/intro_git/)もわかりやすい。

## Gitを始める

### インストール

[こちら](https://git-scm.com/book/ja/v2/使い始める-Gitのインストール)を参照。MacOSXであればデフォルトで入っている。

### 設定

名前とメールアドレスを登録
```sh
git config --global user.name "Akihiro Sanada"
git config --global user.email "akihiro.sanada@teradata.com"
```

設定一覧は下記のコマンドで見ることができる。
```sh
git config --list # 一覧
git config user.name # 指定```

#### (Optional)TeradataのPrivateレポジトリにアクセスするには(※[こちら](https://qiita.com/shizuma/items/2b2f873a0034839e47ce)と同じ内容)

秘密鍵/公開鍵があるか確認。
```sh
ls ~/.ssh # id_rsaなどの名前がついたファイルがあるかどうか確認```

> ない場合は作成。2回目/3回目のパスワードは入れても入れなくても良い。
    ```sh
    ssh-keygen -t rsa```
    パスワードをつけた場合は、ついでにssh-addでパスワードを登録しておくと、毎回聞かれなくて楽。
    ```sh
    ssh-add <作成した秘密鍵のパス>```
    ちなみに.pubがついていない方が秘密鍵で、.pubがついているのは公開鍵。

GitHubにsshの公開鍵(.pubの方)を登録する。
> [こちら](https://github.com/settings/keys)にアクセス。
"Add SSH key"を押す。titleは自分が分かる名前を自由につける。keyに~~.pubの中身を貼り付ける。
>> - ~~.pubのクリップボードへのコピーは以下でできる。あとはペーストすればOK。
```sh
cat <作成した公開鍵のパス> > pbcopy```
ちなみに`>`は左の出力を右の対象に上書きするコマンド。`cat`はファイルの中身を出力するコマンド。

> 登録した公開鍵の横の、Enable SSO(Single Sign On)をクリック。Authorizeを押すと、組織(この場合はTeradata)のPrivateレポジトリにアクセスできるようになる。

sshの設定。  
```
Host github github.com
    HostName github.com
    IdentityFile <秘密鍵のパス>
    User git
```  
を`~/.ssh/config`ファイルに追記。

githubに接続テスト。
```sh
ssh -T github```

githubに接続するときに、基本的にsshを使用するように設定。以下を`~/.gitconfig`に追加。
```
[url "github:"]
    InsteadOf = https://github.com/
    InsteadOf = git@github.com:
```

## gitの全体像イメージ

1. リモートレポジトリ(GitHubなど)をローカルにcloneして、
2. 作業用ブランチ(add_func_1)を作成(branch/checkoutで)、
3. 途中、他のブランチ(add_func_1_fix)を作成して作業したり、作業結果を反映させたり(merge)、
4. 最新のmasterをGitHubから引っ張って更新したり(fetch/pull)して、
5. 作業結果をGitHubにアップロード(push)、
6. 作業結果をレビューしてもらい(pull request)、結果を反映(merge)

ローカルでの作業時には(status, add/rm, commit, rebase, stash, diff, config)などのコマンドを使う

![git_all_image](figures/git_material_figure.jpg)

## clone

リモートレポジトリ(GitHubやBitBucket, AWS CodeCommit, その他のサーバーなど)からレポジトリをコピーしてローカルPCに持ってくる。

レポジトリのフォルダを入れたい場所に移動してから、
```sh
git clone <git@github.com:アカウント/ダウンロードしたいレポジトリ.git>```
例えば、
```sh
git clone git@github.com:ThinkBigAnalytics/ds-japan.git```


## status

gitの状況を確認する。頻繁に打つコマンド。
```sh
git status```

以下のような出力が出る
```
On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

    new file:   README

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  
    modified:   CONTRIBUTING.md
```

- 上記では、今はmasterブランチにいて、READMEファイルの新規作成が"ステージされている"(コミット対象となり、コミットを待っている)状況。  
- 一方、CONTRIBUTING.mdの変更は、まだステージされていない。コミットの対象にはならない。

## add / rm

次のコマンドで、変更/作成/削除をステージできる
```sh
git add CONTRIBUTING.md```

まとめてステージすることも可能
```sh
git add . # 現在のフォルダ以下を再帰的に全部ステージ
git add myfolder # myfolder以下を再帰的に全部ステージ
git add *.py # 現在のフォルダ以下の.pyファイルを全てステージ
git add myfolder/*.py # myfolder以下の.pyファイルを全てステージ
```

ステージされた変更等はrmでステージから外すことができる
```sh
git rm CONTRIBUTING.md```

まとめてステージから外すことも可能
```sh
git rm . # 現在のフォルダ以下を再帰的に全部ステージ
git rm myfolder # myfolder以下を再帰的に全部ステージ
git rm *.py # 現在のフォルダ以下の.pyファイルを全てステージ
git rm myfolder/*.py # myfolder以下の.pyファイルを全てステージ
```

## diff

前回のcommitからの変更を見ることができる。

```sh
git diff # まだステージされていない変更を見る
git diff --staged # ステージされた変更を見る
```

ちなみに、diffで使用するツールを変更したい場合は、git config --global core.editorではなくdiftoolsの設定を変更する必要がある。詳細は[こちら](https://stackoverflow.com/questions/6412516/configuring-diff-tool-with-gitconfig)を参照。

## commit

ステージされた変更等をコミットする。コミットされた変更は記録として残る。
```sh
git commit -m "コミットのメッセージ。~~を追加、など"```

## reset

過去のcommitをresetできる。

```sh
git reset # 何も指定しないと、直近のcommit(HEAD)またはadd/rm動作がリセットされる。
git reset --hard HEAD # 直近のcommit(HEAD)をリセットする。
git reset --hard <ハッシュ番号> # 
git reset --soft HEAD^ # ```

## push

ローカルのレポジトリをリモートレポジトリ(GitHubやBitBucket, CodeCommitなど)にアップロードする。

## fetch / pull

リモートレポジトリの最新情報を取ってくるには
```sh
git fetch```

リモートのmasterブランチが更新されたとする。
ローカルのmasterブランチにその更新を反映させるには、
```sh
git pull origin master```

## branch / checkout

branch一覧を見る  
```sh
git branch```

"add_func_1"というbranchを作成するに切り替える
```sh
git branch add_func_1```

add_func_1というbranchに切り替える
```sh
git checkout add_func_1```

## stash

一時的に、今のブランチの変更内容等を退避させたいときがしばしば発生する。その際に使えるのがstashコマンド。
```sh
git stash```

で現在の作業内容を避難させることができる。避難させたstash一覧は次のようにして確認する。
```sh
git stash ```

避難させると前回のcommitの状態に戻る。

避難させた内容を復活させるには、
```sh
git stash```

## merge

In [ ]:
# mergeの基本

In [ ]:
# mergeの概念

In [ ]:
# conflictの解消

## rebase

ローカルで作業をしているときに、masterが変更されることがよくある。  
そうしたときに便利なのがrebaseである。
```sh
git rebase master add_func_1```

In [ ]:
# rebaseの説明

注意点としては、既にローカルにpushしてしまったブランチについては、混乱が生まれるため、後からrebaseするべきではない。mergeで対応しよう。

## pull request機能

これは利用するサービスによってやり方が異なる。現在はGitHub, BitBucket, CodeCommitともにpull requestに類する機能があるが、昔はCodeCommitはpull requestがなかったりした。

以下ではGitHubのpull request機能について概説する

- pull requestを出したいbranch(下図ではgpuブランチ)に移動して、"new pull request"ボタンを押す

![new_pullreq](figures/20190531_push_new_pullrequest.png)

- コメントを書いて(変更点など)、"create pull request"ボタンを押す。
    - コメントはmarkdownに対応している。絵文字も使える。
- ReviewersやAssigneesを設定できる。
    - コードのレビューしてほしい人を、Reviewersに指定する。指定された人にはメールがGitHubから届く。

![push_pullreq](figures/20190531_open_pullrequest.png)

- 掲示板のようなフォーマットでディスカッションができる。mergeして良いという判断が下されたら、"merge"ボタンを押してmergeする。
- mergeされたブランチはもう必要ないので、タイミングをみて削除しておく。

![discussion](figures/20190531_pullrequest_discussion.png)

conflictが発生している場合は、mergeのときと同様、mergeしたいbranchを編集してconflictを解消する必要がある。  
ローカルで編集してからpushで再度アップロードしてもよいし、web上で編集(resolve conflictボタンを押す)しても良い。

## その他

### ブランチ名をbashに表示させたい

### VSCodeが使いたい場合

vscodeが使いたい場合は、以下で使える。
```sh
git config --global core.editor 'code --wait' # core editorをvscodeに変更
```


diffでも使いたい場合は、`~/.gitconfig`ファイルに以下を追記してから、
```
[diff]
    tool = vscode
[difftool "vscode"]
    cmd = code --wait --diff $LOCAL $REMOTE
```

次のように叩く
```sh
git difftool```